In this file we will prepare our address information and commodity information datasets for evaluation.

In [13]:
import pandas as pd
import re

address_set = pd.read_csv('C://...//address_info.csv', header=None)

Unneccesary columns are dropped

In [14]:
address_set = address_set.drop([3,4,5], axis=1)

We will check if there are coordinates for each record. If some are missing, we may cross-reference using the address.

In [15]:
pattern = '[0-9]{2}.[0-9], [-0-9]'

# Not displayed
#print(address_set[address_set[2].str.contains(pattern) == False])

#Print how many are missing coordinates
print(len(address_set[address_set[2].str.contains(pattern) == False]))

# Copy the records missing coordinates to a new dataframe
coordinate_set = address_set[address_set[2].str.contains(pattern) == False]


21


Only 21 records do not have coordinates. Fortunately it appears that all except for two of the records not containing coordinates have an address (not displayed here).

To get the coordinates we will be utilizing [Google's Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview). Originally the plan was to use the [U.S. Census Bureau's Geocoding Services API](https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.html); however, it did not do a good job of catching any name discrapencies. The basic examples provided in the API documentation have been adapted for this purpose.

In [17]:
import googlemaps
import time
county_list = list()
result_list = list()
gmaps = googlemaps.Client(key='')

delay = 0.1  # Set the delay to 100ms.

for i in coordinate_set[1]:
    # Geocoding an address
    geocode_result = gmaps.geocode(i)
    result_list.append(geocode_result)
    time.sleep(delay)

In [18]:
# Extract the county name for each result
for i in result_list:
    for j in i[0]['address_components']:
        if j['long_name'] and 'County' in j['long_name']:
            county_list.append(j['long_name'])
            
# Extract the coordinates for each result (not needed, but good for completion)
coordinate_list = list()
for i in result_list:
    coordinate_list.append(str(i[0]['geometry']['location']['lat']) + ', ' + str(i[0]['geometry']['location']['lng']))

In [19]:
# Add the counties to a new column
address_set.loc[address_set[2].str.contains(pattern) == False, 3] = county_list

In [20]:
# Add in the newly found coordinates
address_set.loc[address_set[2].str.contains(pattern) == False, 2] = coordinate_list

In [21]:
county_list = list()
result_list = list()
check = list()
gmaps = googlemaps.Client(key='')
ct = 0
delay = 0.1  # Set the delay to 100ms.

for i in address_set.loc[address_set[3].isnull(),1]:
    # Geocoding an address
    geocode_result = gmaps.geocode(i)
    result_list.append(geocode_result)
    check.append
    print(ct)
    ct = ct + 1
    time.sleep(delay)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
county_list = list()
missing_list = list()
c = 0
for i in result_list:
    check = False
    for j in i[0]['address_components']:
        if j['long_name'] and 'County' in j['long_name']:
            county_list.append(j['long_name'])
            check = True
    if check is False:
        missing_list.append(c)
    c = c+ 1

In [23]:
len(missing_list)

22

Twenty three records did not have a county entry- a quick examination showed that all of these had addresses in Baltimore City. "City" is not appended to the city name, which makes it difficult to check for in advance without knowing which cities will be included. This is further complicated as Baltimore County and Baltimore City are different entities. Fortunately there are few records in this situation, so it is easy to check for manually.

In [161]:
#for i in missing_list:
#    result = ""
#    for j in result_list[i][0]['address_components']:
#        result = result + ": " + j['long_name']
#    print(result)

Now we will run through the results again, adding in Baltimore City if there is no county.

In [24]:
county_list = list()
for i in result_list:
    check = False
    for j in i[0]['address_components']:
        if j['long_name'] and 'County' in j['long_name']:
            county_list.append(j['long_name'])
            check = True
    if check is False:
        county_list.append('Baltimore City')

And the counties (or city) are added to the new column.

In [25]:
address_set.loc[address_set[3].isnull(),3] = county_list

Export the results to a csv.

In [32]:
address_set.to_csv('counties.csv')

Now work on the commodities

In [96]:
commodity_set = pd.read_csv('C://...//commodity_list.csv', header=None)

commodity_set

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,1,Hunting,Compost-manure,Hay,Honey,Lamb,Livestock,Meat-fowl,Sheep,Straw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Beef,Bison,Eggs,Fruits,Honey,Vegetables,Conventional,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Apples,Dried-flowers,Dried-herbs,Edible-flowers,Flowers-nurseries,Grapes,Herbs,Lettuce,Nuts-seeds,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Fruits,Vegetables,Conventional,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,344,Christmas-wreaths,Dried-flowers,Perennials-plants,Plants-bedding-etc,Trees-shrubs,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,345,Egg Processing,Grass Fed/Pastured,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,346,Apples,Apricots,Asparagus,Bath-body-products,Beef,Beets,Blackberries,Blueberries,Broccoli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# Drop the ID column
commodity_set = commodity_set.drop([0], axis = 1)

Now we will find out how many unique item entries exist:

In [132]:
commodity_dict = dict()
for i, j in commodity_set.iterrows():
        for x in j:
            if pd.notna(x):
                temp = x
                commodity_dict[temp] = commodity_dict.get(temp, 0) + 1

In [133]:
commodity_dict

{'Hunting': 2,
 'Compost-manure': 8,
 'Hay': 24,
 'Honey': 59,
 'Lamb': 24,
 'Livestock': 17,
 'Meat-fowl': 16,
 'Sheep': 6,
 'Straw': 20,
 'Wool': 10,
 'Conventional': 166,
 'Grass Fed/Pastured': 31,
 'Halal': 2,
 'Customer Pick Up': 51,
 'Beef': 54,
 'Bison': 7,
 'Eggs': 56,
 'Fruits': 114,
 'Vegetables': 145,
 'Apples': 41,
 'Dried-flowers': 13,
 'Dried-herbs': 13,
 'Edible-flowers': 19,
 'Flowers-nurseries': 11,
 'Grapes': 15,
 'Herbs': 27,
 'Lettuce': 60,
 'Nuts-seeds': 4,
 'Pears': 23,
 'Plants-bedding-etc': 41,
 'Radish': 45,
 'Sunflower-seeds': 5,
 'Tomatoes': 77,
 'Wreathes': 3,
 'Local Delivery': 3,
 'Trees-shrubs': 12,
 'Corn Maze': 16,
 'Arugula': 30,
 'Beets': 52,
 'Cantaloupes': 53,
 'Cucumbers': 72,
 'Drybeans': 8,
 'Eggplant': 60,
 'Garlic': 38,
 'Gourds': 39,
 'Green-beans': 60,
 'Green-onions': 44,
 'Green-peppers': 71,
 'Hot-peppers': 49,
 'Melons': 50,
 'Okra': 37,
 'Onions': 53,
 'Peaches': 44,
 'Perennials-plants': 30,
 'Potatoes': 55,
 'Pumpkin-patches': 18,
 'Pu

In [134]:
len(commodity_dict)

178